# Notebook for using model

Import Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms
from torchsummary import summary

import os
import argparse

from models.dropoutresnet import DropoutResNet18

Training Function

In [2]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    # switch to train mode
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    # start training epoch
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        # move to device
        inputs, targets = inputs.to(device), targets.to(device)
        
        # forward pass
        optimizer.zero_grad()
        outputs = net(inputs)
        
        # backpropogate
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        # calculate loss and accuracy
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    print('Loss: %.3f | Acc: %.3f%% (%d/%d)' % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

Test Function

In [3]:
def test(epoch):
    global best_acc
    # switch to eval mode
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        # start test epoch
        for batch_idx, (inputs, targets) in enumerate(testloader):
            # move to device
            inputs, targets = inputs.to(device), targets.to(device)
            
            # forward pass
            outputs = net(inputs)
            
            # calculate loss and accuracy
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

        print('Loss: %.3f | Acc: %.3f%% (%d/%d)' % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
        
    # Save checkpoint.
    acc = 100.*correct/total
    
    # Save best epoch checkpoint
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/final.pth')
        best_acc = acc
    
    # Save model
    if epoch == 199:
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        torch.save(state, './checkpoint/final.pth')

Loading variables for training

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

Data Preprocessing

In [5]:
print('==> Preparing data..')

# Data augmentation and normalization for training
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

# Apply Transformation
trainset = torchvision.datasets.CIFAR10(
    root='./deep-learning-mini-project-spring-24-nyu/cifar-10-python', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./deep-learning-mini-project-spring-24-nyu/cifar-10-python', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

# List of classes
classes = ('plane', 'car', 'bird', 'cat', 'deer',
        'dog', 'frog', 'horse', 'ship', 'truck')

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


Initializing Model and training tools

In [6]:
print('==> Building model..')
# Initialize the model
net = DropoutResNet18(0.4)
net = net.to(device)

# If the model is on cuda, use DataParallel
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

# Print out the model parameters
summary(net, (3,32,32))

==> Building model..
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
ModifiedBasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
ModifiedBasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13           [-1, 64, 32, 32]          36,864
      BatchNorm2d

Running Training and Inference

In [7]:
for epoch in range(start_epoch, 200):
    train(epoch)
    test(epoch)
    scheduler.step()


Epoch: 0
Loss: 2.029 | Acc: 24.178% (12089/50000)
Loss: 2.346 | Acc: 22.850% (2285/10000)
Saving..

Epoch: 1
Loss: 1.587 | Acc: 40.508% (20254/50000)
Loss: 1.422 | Acc: 46.580% (4658/10000)
Saving..

Epoch: 2
Loss: 1.353 | Acc: 50.404% (25202/50000)
Loss: 1.477 | Acc: 48.010% (4801/10000)
Saving..

Epoch: 3
Loss: 1.173 | Acc: 57.910% (28955/50000)
Loss: 1.882 | Acc: 45.350% (4535/10000)

Epoch: 4
Loss: 1.005 | Acc: 64.542% (32271/50000)
Loss: 1.056 | Acc: 64.190% (6419/10000)
Saving..

Epoch: 5
Loss: 0.872 | Acc: 69.550% (34775/50000)
Loss: 0.871 | Acc: 69.950% (6995/10000)
Saving..

Epoch: 6
Loss: 0.761 | Acc: 73.698% (36849/50000)
Loss: 0.825 | Acc: 72.490% (7249/10000)
Saving..

Epoch: 7
Loss: 0.675 | Acc: 76.862% (38431/50000)
Loss: 0.977 | Acc: 67.830% (6783/10000)

Epoch: 8
Loss: 0.619 | Acc: 78.928% (39464/50000)
Loss: 0.961 | Acc: 70.840% (7084/10000)

Epoch: 9
Loss: 0.566 | Acc: 80.848% (40424/50000)
Loss: 0.698 | Acc: 76.480% (7648/10000)
Saving..

Epoch: 10
Loss: 0.538 | Ac

In [8]:
print(best_acc)

95.74
